In [7]:
data_dir = "/Volumes/Data/Research/Healthy Brain Network/ActiGraph/gt3x_data"
subdirs = ["in process", "processed"]

In [47]:
from datetime import datetime, timedelta
import os
import numpy as np
import pandas as pd
import zipfile

In [33]:
def read_zips(data_dir, subdirs):
    """
    Function to read in all specified raw csvs.
    
    Parameters
    ----------
    data_dir : string
        parent directory
        
    subdirs : list of strings
        subdirectories
        
    Returns
    -------
    dfs : list of 3-tuples of strings, DataFrames, and strings
        ([0], : string
            filename
        [1], : Dataframe
            RAW csv file exported from ActiLife
        [2]) : string
            device serial number
    """
    dfs = list()
    for subdir in subdirs:
        sd = os.path.join(data_dir, subdir)
        for file in os.listdir(sd):
            if file.endswith(".gt3x"):
                zp = os.path.join(sd, file)
                zdict = {}
                try:
                    with zipfile.ZipFile(zp) as z:
                        with z.open("info.txt", "r") as f:
                            for line in f:
                                try:
                                    key, value = line.decode("UTF-8").split(":")
                                    zdict[key] = value[1:-2]
                                except:
                                    continue
                    dfs.append(zdict)
                except:
                    print(zp)
    return(dfs)


def ticks_to_datetime(ticks):
    """
    Function to convert .NET ticks to datetime
    
    Parameter
    ---------
    ticks : int
    
    Returns
    -------
    dt : datetime
    """
    return(datetime(1, 1, 1) + timedelta(microseconds = int(x)//10))

In [34]:
z = read_zips(data_dir, subdirs)

/Volumes/Data/Research/Healthy Brain Network/ActiGraph/gt3x_data/in process/5783223 (2017-09-14).gt3x


In [39]:
df = pd.DataFrame.from_dict(z)

In [53]:
df["Start Date"].map(lambda x: )

0    2017-08-03 12:25:00
1    2017-08-17 12:34:00
2    2017-08-17 12:34:00
3    2017-08-30 17:51:00
4    2017-08-02 17:54:00
5    2017-08-23 13:26:00
6    2017-08-24 14:17:00
7    2017-07-27 16:39:00
8    2017-08-24 13:19:00
9    2017-08-07 17:04:00
10   2017-08-04 18:06:00
11   2017-08-31 12:32:00
12   2017-08-09 13:56:00
13   2017-09-01 13:23:00
14   2017-08-10 18:21:00
15   2017-08-11 12:29:00
16   2017-08-04 13:14:00
17   2017-08-30 13:00:00
18   2017-08-21 13:15:00
19   2017-08-09 17:50:00
20   2017-08-22 17:00:00
21   2017-08-10 12:52:00
22   2017-08-14 16:29:00
23   2017-08-07 17:13:00
24   2017-08-12 12:52:00
25   2017-08-30 12:01:00
26   2017-08-07 12:20:00
27   2017-08-05 15:32:00
28   2017-08-31 16:21:00
29   2017-07-28 17:20:00
30   2017-07-27 12:13:00
31   2017-08-08 16:42:00
32   2017-09-06 17:28:00
33   2017-08-21 18:11:00
34   2017-08-01 12:09:00
35   2017-08-31 12:36:00
36   2017-07-28 12:45:00
37   2017-08-10 18:10:00
38   2017-08-18 11:41:00
39   2017-08-24 12:25:00
